It important to keep backing up your data in external memory devices at the regular interval. Since i don't have a lot of multimedia data, i use a 256 GB mirco sd card for my backups .But usually we don't have enough to time to go through all of our files and copy only the files that are modified since the last backup. Plus, there are a few file/folder that i don't need to backup, but i still end up copying them anyways. This is both, time consuming and space consuming. 

A solution is to automate the backup process to make it more streamlined and less space and time consuming. One way to achieve this , which i am going to implement here is as follows. Keep track of the last time you took backup. specify all the folder that needs to be copied. go over through the folder and identify files that were modified after the last backup date. copy all the updated files in the new folder  in the external memory device. upadate the last backup time stamp and we are done 

**Prerequisites**

- understand how external devices are mounted in the the file system ( i am using macOS)
- understand how to access interal and external files in Python
- how to access file metadata in pyhon
- create a config file, read/update the file after every backup

**todo later**
- analytics : how many new files updates, compare with older backups .... 

----

Mount/unmount in macOS
- list all drives : `diskutil list`
- to mount a disk
    - find the identifier of the disk to mount: `diskutil list | grep bakup_disk`
    - mount it : `diskutil mount /dev/disk_identifer`
- to unmount a specific dive : `diskutil unmount /dev/disk1s2
- how to access the disk mounted
    - find the path where was the disk was mounted : `mount | grep disk_name`
    - then `cd /Volumes/disk_name` (or where ever the disk was mounted)

In [53]:
# imports 
import os 
import json
from datetime import datetime
import time
import shutil

In [3]:
def getLastBackupTime(fileName="backUpLogs"):
    '''return the datetime of the last time backup was done
    by reading the log file. when run first time, creates a blank log file
    '''
    file = fileName+".json"
    if not os.path.exists(file):
        print("Log file does not exists. Creating one")
        blank = {"logs":[]}
        with open(file, 'w') as json_file:
            json.dump(blank, json_file)
        return None    
    backupfile =  open(file,"r")
    backups = json.load(backupfile)
    backupfile.close()
    if len(backups["logs"]) is 0:
        return None
    else :
        date = backups["logs"][len(backups["logs"])-1]["date"]
        return date
        
print(getLastBackupTime())

Log file does not exists. Creating one
None


In [4]:
def getCurrentDate():
    now = datetime.now()
    dtStr = now.strftime("%d-%m-%Y %H:%M:%S")
    return dtStr

In [78]:
def updateLogFile(newRecord,fileName="backUpLogs"):
    ''' To update the log file after successful backup '''
    fileN = fileName+".json"
    backupfile =  open(fileN,"r")
    backups = json.load(backupfile)
    backupfile.close()
    newRecord["date"] = time.time() # getCurrentDate()
    backups["logs"].append(newRecord)
    backups_json = json.dumps(backups,indent = 4)
    # print(backups_json)
    backupfile1 =  open(fileN,"w")
    backupfile1.write(backups_json)
    backupfile1.close()
    print("Saved")

# updateLogFile({})

Saved


In [6]:
def setCWDtoHome(username):
    os.chdir("/Users/"+username)

In [120]:
config = {
    "userName":"shubhvjain",
    "logFileName":"dataBk",
    "folders" : [
        {
            "path":"sample1",
            "ignore":["sample2"],
            "ignoreFile":[]
        }
        ,
        {
            "path":"nb",
            "ignore":[".git","node_modules"],
            "ignoreFile":[".DS_Store"]
        }
    ] ,
    "destPath":"/Users/shubhvjain/backupSh", # root of destination path where the backup will be set 
    "backUpLogFile":True # to also take a backup of the backup log file 
}
lastTime = getLastBackupTime(config["logFileName"])
setCWDtoHome(config["userName"])
#print("cwd:",os.getcwd())
#print(lastTime)
#print("curr time",time.time())

newBackup = {
    "logs" :[],
    "config" : config,
    "stats":{"new":0,"copied":0}
}       

def addToLog(*args):
    result = ""
    for x in args:
        result += " "+x
    newBackup["logs"].append({"data":time.time(),"msg": result })

In [136]:
for folder in config["folders"]:
    # addToLog("Reading folder : ",folder["path"])
    # print("Path : ",folder["path"])
    for root, dirs, files in os.walk(folder["path"]):
        for name in files:
            pt = os.path.join(root, name)
            pathStr = str(pt)
            parts = pathStr.split("/")
            
            copyRequired = False # to check if the file is not inside an ignored folder
            
            if folder["ignore"] :
                ignoreFile =  any(item in parts for item in folder["ignore"])
                if ignoreFile is False:
                    copyRequired = True
            else :
                copyRequired = True
            
            
            if name in folder["ignoreFile"]:
                copyRequired = False
            
                        
            if copyRequired:
                # check if file already exists at destination 
                destPath = os.path.join(config["destPath"],pt)
                part2 = destPath.split("/")
                if len(part2) > 1 :
                    part2.pop()
                dirPath =  '/'.join(part2)  
                fileExists = os.path.exists(destPath)
                fullSrcPath = os.path.join(os.getcwd(),pt)
                if fileExists :
                    # check if the file is modified since the last update
                    fileSame = round(os.path.getmtime(fullSrcPath)) == round(os.path.getmtime(destPath))
                    if not fileSame:
                        #print(fullSrcPath)
                        print("file  to be copied")
                        cmd2 = 'cp -p "'+fullSrcPath+'"  "'+ destPath+'"'
                        a1 = os.system(cmd2)
                else:
                    if pt is not destPath:
                        cmd1 = 'mkdir -p "'+ dirPath+'"'
                        cmd2 = 'cp -p "'+fullSrcPath+'"  "'+ destPath+'"'
                        a1 = os.system(cmd1)
                        a2 = os.system(cmd2)

# print(newBackup) 
updateLogFile(newBackup,config["logFileName"])
cmd3 = 'cp -p "'+os.path.join(os.getcwd(),config["logFileName"]+".json")+'"  "'+ config["destPath"]+'"'
a1 = os.system(cmd3)
print(a1)

Saved
0
